# 第六章 问答

 - [一、引言](#一、引言)
 - [二、环境配置](#二、环境配置)
 - [三、加载向量数据库](#三、加载向量数据库)
 - [四、构造检索式问答连](#四、构造检索式问答连)
 - [五、深入探究检索式问答链](#五、深入探究检索式问答链)
     - [5.1 基于模板的检索式问答链](#5.1-基于模板的检索式问答链)
     - [5.2 基于 MapReduce 的检索式问答链](#5.2-基于-MapReduce-的检索式问答链)
     - [5.3 基于 Refine 的检索式问答链](#5.3-基于-Refine-的检索式问答链)
 - [六、实验：状态记录](#六、实验：状态记录)


## 一、引言


在上一章，我们已经讨论了如何检索与给定问题相关的文档。下一步是获取这些文档，拿到原始问题，将它们一起传递给语言模型，并要求它回答这个问题。在本课程中，我们将详细介绍这一过程，以及完成这项任务的几种不同方法。

我们已经完成了整个存储和获取，获取了相关的切分文档之后，现在我们需要将它们传递给语言模型，以获得答案。这个过程的一般流程如下：首先问题被提出，然后我们查找相关的文档，接着将这些切分文档和系统提示一起传递给语言模型，并获得答案。

默认情况下，我们将所有的文档切片都传递到同一个上下文窗口中，即同一次语言模型调用中。然而，有一些不同的方法可以解决这个问题，它们都有优缺点。大部分优点来自于有时可能会有很多文档，但你简单地无法将它们全部传递到同一个上下文窗口中。MapReduce、Refine 和 MapRerank 是三种方法，用于解决这个短上下文窗口的问题。我们将在该课程中进行简要介绍。

## 二、环境配置

配置环境方法同前，此处不再赘述

In [2]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']


在2023年9月2日之后，GPT-3.5 API 会进行更新，因此此处需要进行一个时间判断

In [3]:
# import datetime
# current_date = datetime.datetime.now().date()
# if current_date < datetime.date(2023, 9, 2):
#     llm_name = "gpt-3.5-turbo-0301"
# else:
# llm_name = "gpt-3.5-turbo"
# print(llm_name)

## 三、加载向量数据库

In [4]:
# 加载在之前已经进行持久化的向量数据库
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


persist_directory = 'docs/chroma/cs229_lectures/'
embedding_model = OpenAIEmbeddings()
vector_db = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding_model)

# 可以看见包含了我们之前进行分割的209个文档
print(vector_db._collection.count())

209


我们可以测试一下对于一个提问进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回给你 k 个文档。

In [5]:
question = "What are major topics for this class?"
docs = vector_db.similarity_search(question,k=3)
print(len(docs))
for doc in docs:
    print(doc)
print()


question = "这节课的主要话题是什么"
docs = vector_db.similarity_search(question,k=3)
print(len(docs))

3
page_content="statistics for a while or maybe algebra, we'll go over those in the discussion sections as a \nrefresher for those of you that want one.  \nLater in this quarter, we'll also use the disc ussion sections to go over extensions for the \nmaterial that I'm teaching in the main lectur es. So machine learning is a huge field, and \nthere are a few extensions that we really want  to teach but didn't have time in the main \nlectures for." metadata={'page': 8, 'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf'}
page_content="statistics for a while or maybe algebra, we'll go over those in the discussion sections as a \nrefresher for those of you that want one.  \nLater in this quarter, we'll also use the disc ussion sections to go over extensions for the \nmaterial that I'm teaching in the main lectur es. So machine learning is a huge field, and \nthere are a few extensions that we really want  to teach but didn't have time in the main \nlectures for." metadata={'page'

## 四、构造检索式问答连

基于 LangChain，我们可以构造一个使用 GPT3.5 进行问答的检索式问答链，这是一种通过检索步骤进行问答的方法。我们可以通过传入一个语言模型和一个向量数据库来创建它作为检索器。然后，我们可以用问题作为查询调用它，得到一个答案。

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat_llm = ChatOpenAI(
    temperature = 0.0,
)
# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vector_db.as_retriever()
)

In [7]:
# 可以以该方式进行检索问答
question = "What are major topics for this class?"
result = qa_chain({"query": question})

print(result)
print(result["result"])

# # 可以以该方式进行检索问答
# question = "这节课的主要话题是什么"
# result = qa_chain({"query": question})

# print(result["result"])

{'query': 'What are major topics for this class?', 'result': 'The major topics for this class are machine learning and its various extensions.'}
The major topics for this class are machine learning and its various extensions.


## 五、深入探究检索式问答链

通过上述代码，我们可以实现一个简单的检索式问答链。接下来，让我们深入其中的细节，看看在这个检索式问答链中，LangChain 都做了些什么。


### 5.1 基于模板的检索式问答链


我们首先定义了一个提示模板。它包含一些关于如何使用下面的上下文片段的说明，然后有一个上下文变量的占位符。

In [9]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)



# # Build prompt
# Chinese_template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
# {context}
# 问题：{question}
# 有用的回答："""
# QA_CHAIN_PROMPT = PromptTemplate.from_template(Chinese_template)


# Run chain
qa_chain = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vector_db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "Is probability a class topic?"
result = qa_chain({"query": question})
print(result["result"])
print()

# 中文版
question = "机器学习是其中一节的话题吗"
result = qa_chain({"query": question})
print(result["result"])
print(result["source_documents"][0])

print()

Yes, probability is a topic that will be covered in the class. Thanks for asking!

Yes, machine learning is one of the topics covered in this class. Thanks for asking!
page_content="So in this class, we've tried to convey to you a broad set of principl es and tools that will \nbe useful for doing many, many things. And ev ery time I teach this class, I can actually \nvery confidently say that af ter December, no matter what yo u're going to do after this \nDecember when you've sort of completed this  class, you'll find the things you learn in \nthis class very useful, and these things will be useful pretty much no matter what you end \nup doing later in your life.  \nSo I have more logistics to go over later, but let's say a few more words about machine \nlearning. I feel that machine learning grew out of  early work in AI, early work in artificial \nintelligence. And over the last — I wanna say last 15 or last 20 years or so, it's been viewed as a sort of growing new capability for co

这种方法非常好，因为它只涉及对语言模型的一次调用。然而，它也有局限性，即如果文档太多，可能无法将它们全部适配到上下文窗口中。我们可以使用另一种技术来对文档进行问答，即MapReduce技术。

### 5.2 基于 MapReduce 的检索式问答链

在 MapReduce 技术中，首先将每个独立的文档单独发送到语言模型以获取原始答案。然后，这些答案通过最终对语言模型的一次调用组合成最终的答案。虽然这样涉及了更多对语言模型的调用，但它的优势在于可以处理任意数量的文档。


In [11]:
#import os
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
#os.environ["LANGCHAIN_API_KEY"] = "..."

os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "langchainPlayGround"
os.environ["WANDB_NOTEBOOK_NAME"] = "QA"

In [15]:
qa_chain_mr = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vector_db.as_retriever(),
    chain_type="map_reduce"
)
question = "Is probability a class topic?"
result = qa_chain_mr({"query": question})
print(result["result"])
print()

qa_chain_mr = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vector_db.as_retriever(),
    chain_type="map_reduce"
)
# 中文版
question = "概率论是其中一节的话题吗"
result = qa_chain_mr({"query": question})
print(result["result"])
print()

Yes, probability is mentioned as a prerequisite for the class. The instructor assumes familiarity with basic probability and statistics.



Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


根据提供的文本部分，无法确定概率论是否是课程的一个主题。



当我们将之前的问题通过这个链进行运行时，我们可以看到这种方法的两个问题。第一，速度要慢得多。第二，结果实际上更差。根据给定文档的这一部分，对这个问题并没有明确的答案。这可能是因为它是基于每个文档单独回答的。因此，如果信息分布在两个文档之间，它并没有在同一上下文中获取到所有的信息。

我们可导入上述环境变量，然后探寻 MapReduce 文档链的细节。例如，上述演示中，我们实际上涉及了四个单独的对语言模型的调用。在运行完每个文档后，它们会在最终链式中组合在一起，即Stuffed Documents链，将所有这些回答合并到最终的调用中。

### 5.3 基于 Refine 的检索式问答链

我们可以类似地设置链式类型为Refine。这是一种新的链式类型。Refine 文档链类似于 MapReduce 链，对于每一个文档，会调用一次 LLM，但有所改进的是，我们每次发送给 LLM 的最终提示是一个序列，这个序列会将先前的响应与新数据结合在一起，并请求得到改进后的响应。因此，这是一种类似于 RNN 的概念，我们增强了上下文，从而解决信息分布在不同文档的问题。

In [18]:
qa_chain_r = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vector_db.as_retriever(),
    chain_type="refine"
)
question = "Is probability a class topic?"
result = qa_chain_r({"query": question})
print(result["result"])
print()

qa_chain_r = RetrievalQA.from_chain_type(
    chat_llm,
    retriever=vector_db.as_retriever(),
    chain_type="refine"
)
question = "概率论是其中一节的话题吗"
result = qa_chain_r({"query": question})
print(result["result"])

Based on the additional context provided, it is still not explicitly mentioned whether probability is a specific topic covered in the class. However, the instructor does mention going over statistics and algebra in the discussion sections as a refresher, which suggests that probability may be included as part of the statistics component. Additionally, the instructor mentions using the discussion sections to cover extensions for the material taught in the main lectures, so it is possible that probability could be discussed as an extension topic in relation to machine learning. Therefore, while probability is not explicitly confirmed as a class topic, it is likely to be covered to some extent within the broader context of statistics and machine learning.

根据新的上下文信息，可以确认概率论是其中一节的话题之一。在教授的讲话中，他提到了对基本概率和统计的熟悉，并且假设学生们已经学过类似于斯坦福大学的统计学课程。这暗示了概率论是课程中的一部分，因为概率论是统计学的基础。因此，概率论可能是其中一节的话题之一。


你会注意到，这个结果比MapReduce链的结果要好。这是因为使用Refined Chain允许你逐个地组合信息，实际上比MapReduce链鼓励更多的信息传递。

## 六、实验：状态记录

让我们在这里做一个实验。

我们将创建一个QA链，使用默认的stuff。让我们问一个问题，概率论是课程的主题吗？它会回答，概率论应该是先决条件。我们将追问，为什么需要这些先决条件？然后我们得到了一个答案。这门课的先决条件是假定具有计算机科学和基本计算机技能和原理的基本知识。这与之前问有关概率的问题毫不相关。

说明chain 的调用是没有状态的，因此模型无法了解下上下文。

In [22]:
from langchain.llms import OpenAI

llm = OpenAI(
    temperature = 0.0 
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_db.as_retriever()
)

question = "Is probability a class topic?"
result = qa_chain({"query": question})
print(result["result"])
print()


question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
print(result["result"])
print()


question = "概率论是这节课的一个内容吗"
result = qa_chain({"query": question})
print(result["result"])
print()


question = "为什么需要具备这些知识"
result = qa_chain({"query": question})
print(result["result"])
print()

 Yes, the instructor assumes familiarity with basic probability and statistics and will go over it in the discussion sections as a refresher for those who need it.

 Those prerequisites are needed because the class will involve programming in MATLAB or Octave, and will require knowledge of basic probability and statistics, as well as basic linear algebra.

 是的，概率论是这节课的一个内容。老师假设所有学生都熟悉基本的概率和统计，并且在讨论部分会提供复习课程。

 The speaker is trying to convey that having knowledge of machine learning principles and tools will be useful for many applications, such as reading handwritten characters and autonomous flight.



基本上，我们使用的链式（chain）没有任何状态的概念。它不记得之前的问题或之前的答案。为了实现这一点，我们需要引入内存，这是我们将在下一节中讨论的内容。